In [ ]:
from collections import namedtuple
from hyperopt import fmin, tpe, hp
from hyperopt.mongoexp import MongoTrials

from ddpg_agent.QuadcopterEvaluator import noise_evaluator, evaluator

EXP_NUM=1

trials = MongoTrials('mongo://localhost:27017/hyperopt/jobs', exp_key='quadcopter_noise_exp%3i'%EXP_NUM)

noise_space = namedtuple('NoiseParams',['ou_mu','ou_theta','ou_sigma','n_episodes','eps'])(
        0,
        1,
        hp.uniform('ou_sigma',0,.25),
        100,
        hp.uniform('eps',20,1000),
    )

best_noise = fmin(noise_evaluator, noise_space, trials=trials, algo=tpe.suggest, max_evals=5 )

print(best_noise)

# space = hp.choice('activation_fn_actor',
#     [
# #         ('case 1', 1 + hp.lognormal('c1', 0, 1)),
#         ('tanh', hp.uniform('', , ))
#         ('sigmoid', hp.uniform('c2', -10, 10))
#     ])
# space = (hp.uniform('eps',.5,1.5),hp.uniform('ou_theta',.005,.15))

# best = fmin(evaluate_hyperparams, space, trials=trials, algo=tpe.suggest, max_evals=27 )

# print(best)


over-writing old domain trials attachment


67it [01:07,  1.01s/it, best loss: -56.33024386576488]                      


In [ ]:
trials.best_trial